In [1]:
if (!requireNamespace("BiocManager", quietly=TRUE)) 
    install.packages("BiocManager") 

BiocManager::install("ANCOMBC")
#BiocManager::install(version = "3.21", ask = FALSE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'ANCOMBC'

also installing the dependencies ‘rootSolve’, ‘lmom’, ‘gmp’, ‘Rmpfr’, ‘ECOSolveR’, ‘scs’, ‘osqp’, ‘expm’, ‘Exact’, ‘gld’, ‘gsl’, ‘TH.data’, ‘CVXR’, ‘DescTools’, ‘energy’, ‘gtools’, ‘lmerTest’, ‘multcomp’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'boot', 'cards', 'checkmate', 'crosstalk', 'curl', 'emmeans',
  'evaluate', 'gmm', 'gtsummary', 'hardhat', 'later', 'Matrix', 'rprojroot',
  'rvest', 'sf', 'V8', 'xfun', 'xml2'



In [3]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1013. Microbiome Peanut Allergy/P1013.3. Analyses/P1013.3.1. Data Processing/Output'
cur_date = '083025'

library(readxl)
library(openxlsx)
library(tidyverse)
library(factoextra)
library(ANCOMBC) 

# reading in files
saliva_df = data.frame(read_excel("Input/Saliva_Diversity_082925.xlsx", sheet = 2)) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE))
stool_df = data.frame(read_excel("Input/Stool_Diversity_082925.xlsx", sheet = 2)) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa

Warning message:
“There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Age = as.numeric(Age)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning.”
Warning message:
“There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Age = as.numeric(Age)`.
Caused by warning:
! NAs introduce

In [4]:
head(saliva_df)

,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Peptococcaceae.g__Peptococcus.s__sp._HMT_167,⋯,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.s__sp._HMT_219,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.__,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__wadei,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__shahii,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_498,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_215,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_392,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_212,k__Bacteria.p__Gracilibacteria_.GN02..c__Gracilibacteria_.GN02._.C.1..o__Gracilibacteria_.GN02._.O.1..f__Gracilibacteria_.GN02._.F.1..g__Gracilibacteria_.GN02._.G.1..s__bacterium_HMT_872,k__Bacteria.p__Firmicutes.c__Bacilli.o__Lactobacillales.f__Lactobacillaceae.g__Lacticaseibacillus.__
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS01baseline,1,Pre_1,Pre,P,F,3.838356,41.00,High,0,⋯,0,0,0,0,0,18,15,42,0,0
2,FS0236mo,2,Post_2,Post,A,DS,1.873973,14.20,Low,59,⋯,0,0,0,0,0,532,30,27,0,0
3,FS02baseline,2,Pre_2,Pre,A,DS,1.873973,14.20,Low,0,⋯,0,198,0,0,0,172,0,64,0,0
4,FS0436mo,3,Post_3,Post,P,F,3.715068,67.40,High,0,⋯,0,0,0,0,0,0,0,0,15,0
5,FS04baseline,3,Pre_3,Pre,P,F,3.715068,67.40,High,0,⋯,0,0,0,0,0,0,0,8,0,0
6,FS0536mo,4,Post_4,Post,A,DS,3.991781,9.59,Low,24,⋯,0,0,0,0,0,676,0,207,0,0


words.

In [5]:
data(QMP, package = "ANCOMBC")
set.seed(123)
n = 150
d = ncol(QMP)
diff_prop = 0.1
lfc_cont = 1
lfc_cat2_vs_1 = -2
lfc_cat3_vs_1 = 1

# Generate the true abundances
abn_data = sim_plnm(abn_table = QMP, taxa_are_rows = FALSE, prv_cut = 0.05, 
                    n = n, lib_mean = 1e8, disp = 0.5)
log_abn_data = log(abn_data + 1e-5)
rownames(log_abn_data) = paste0("T", seq_len(d))
colnames(log_abn_data) = paste0("S", seq_len(n))

# Generate the sample and feature meta data
# Sampling fractions are set to differ by batches
smd = data.frame(samp_frac = log(c(runif(n/3, min = 1e-4, max = 1e-3),
                                   runif(n/3, min = 1e-3, max = 1e-2),
                                   runif(n/3, min = 1e-2, max = 1e-1))),
                 cont_cov = rnorm(n),
                 cat_cov = as.factor(rep(seq_len(3), each = n/3)))
rownames(smd) = paste0("S", seq_len(n))
                      
fmd = data.frame(taxon = paste0("T", seq_len(d)),
                 seq_eff = log(runif(d, min = 0.1, max = 1)),
                 lfc_cont = sample(c(0, lfc_cont), 
                                   size = d,
                                   replace = TRUE,
                                   prob = c(1 - diff_prop, diff_prop)),
                 lfc_cat2_vs_1 = sample(c(0, lfc_cat2_vs_1), 
                                        size = d,
                                        replace = TRUE,
                                        prob = c(1 - diff_prop, diff_prop)),
                 lfc_cat3_vs_1 = sample(c(0, lfc_cat3_vs_1), 
                                        size = d,
                                        replace = TRUE,
                                        prob = c(1 - diff_prop, diff_prop))) %>%
    mutate(lfc_cat3_vs_2 = lfc_cat3_vs_1 - lfc_cat2_vs_1)

# Add effect sizes of covariates to the true abundances
smd_dmy = model.matrix(~ 0 + cont_cov + cat_cov, data = smd)
log_abn_data = log_abn_data + outer(fmd$lfc_cont, smd_dmy[, "cont_cov"] )
log_abn_data = log_abn_data + outer(fmd$lfc_cat2_vs_1, smd_dmy[, "cat_cov2"])
log_abn_data = log_abn_data + outer(fmd$lfc_cat3_vs_1, smd_dmy[, "cat_cov3"])

# Add sample- and taxon-specific biases
log_otu_data = t(t(log_abn_data) + smd$samp_frac)
log_otu_data = log_otu_data + fmd$seq_eff
otu_data = round(exp(log_otu_data))

The number of taxa after filtering is: 91



In [7]:
head(otu_data)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,⋯,S141,S142,S143,S144,S145,S146,S147,S148,S149,S150
T1,1,0,1,3,2,0,1,0,0,3,⋯,142,2,208,232,105,64,317,41,320,14
T2,102,1,156,535,178,0,99,21,28,400,⋯,41555,940,69154,150977,68450,40299,258647,23841,79187,7491
T3,0,0,0,0,0,0,0,0,0,0,⋯,17,0,22,23,10,7,32,4,35,2
T4,4,0,7,23,11,0,6,1,1,19,⋯,913,12,1364,1673,767,465,2340,293,2036,96
T5,38,0,65,214,90,0,52,7,9,173,⋯,7679,118,11913,17441,7893,4766,26070,2888,16343,975
T6,15,0,26,85,38,0,23,3,3,70,⋯,3302,45,4985,6334,2864,1756,8992,1100,7295,364


In [8]:
head(smd)

,samp_frac,cont_cov,cat_cov
,<dbl>,<dbl>,<fct>
S1,-7.046117,-0.05133271,1
S2,-8.183509,-1.30178093,1
S3,-7.369319,-1.20561056,1
S4,-7.796003,0.11530542,1
S5,-7.403773,0.78781710,1
S6,-8.488528,-0.64191981,1


In [ ]:
?

In [6]:
set.seed(123)
output = ancombc2(data = otu_data, meta_data = smd, 
                  fix_formula = "cont_cov + cat_cov", rand_formula = NULL,
                  p_adj_method = "holm", pseudo_sens = TRUE,
                  prv_cut = 0.10, lib_cut = 1000, s0_perc = 0.05,
                  group = "cat_cov", struc_zero = FALSE, neg_lb = FALSE,
                  alpha = 0.05, n_cl = 2, verbose = TRUE,
                  global = FALSE, pairwise = TRUE, 
                  dunnet = FALSE, trend = FALSE,
                  iter_control = list(tol = 1e-5, max_iter = 20, 
                                      verbose = FALSE),
                  em_control = list(tol = 1e-5, max_iter = 100),
                  lme_control = NULL, 
                  mdfdr_control = list(fwer_ctrl_method = "holm", B = 100), 
                  trend_control = NULL)

res_prim = output$res
res_pair = output$res_pair

Checking the input data type ...

The input data is of type: matrix

The imported data is in a generic 'matrix'/'data.frame' format.

PASS

Checking the sample metadata ...

The specified variables in the formula: cont_cov, cat_cov

The available variables in the sample metadata: samp_frac, cont_cov, cat_cov

PASS

Checking other arguments ...

The number of groups of interest is: 3

The sample size per group is: 1 = 50, 2 = 50, 3 = 50

PASS

Obtaining initial estimates ...

Estimating sample-specific biases ...

ANCOM-BC2 primary results ...

ANCOM-BC2 multiple pairwise comparisons ...

Conducting sensitivity analysis for pseudo-count addition to 0s ...
For taxa that are significant but do not pass the sensitivity analysis,
they are marked in the 'passed_ss' column and will be treated as non-significant in the 'diff_robust' column.
For detailed instructions on performing sensitivity analysis, please refer to the package vignette.

